In [58]:
!python -m pip install langchain langchain-core langchain-community langchain-experimental langchain-exa --quiet
!python -m pip install -U langchain-google-genai --quiet


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [59]:
model_name = "gemini-2.0-flash"

In [60]:
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000/")
mlflow.set_experiment("gcp-testing-sagar-assignment1")
mlflow.gemini.autolog()

In [61]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

model = init_chat_model(model_name, model_provider="google_genai")
system_prompt = "What is the stock code for given company? Only give stock code and don't generate any other thing"
prompt_temp = ChatPromptTemplate([("system",system_prompt),("user","{company}")])
prompt = mlflow.genai.register_prompt(
    name="gcp-sagar-assignment1",
    template=prompt_temp,
)
analyze_chain = prompt_temp | model | parser
print("Please enter Company name as an input on the top bar")
company_name = input("Enter a company name")
stock_code = analyze_chain.invoke({"company":company_name})
mlflow.log_param("Stock Code", stock_code)
print(stock_code)

2025/09/13 19:06:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: gcp-sagar-assignment1, version 3


Please enter Company name as an input on the top bar
GOOG


In [62]:
from langchain_exa import ExaSearchResults
import os
from dotenv import load_dotenv
load_dotenv()
#Initialize the ExaSearchResults tool
search_tool = ExaSearchResults(exa_api_key=os.getenv('EXA_API_KEY'))


search_results = search_tool._run(
    query=f"Give the detailed news about the company with stock code {stock_code}",
    num_results=50,  # Number of results (1-100)
    type="auto",  # Can be "neural", "keyword", or "auto"
    livecrawl="always",  # Can be "always", "fallback", or "never"
    text_contents_options={"max_characters": 20000},  # Limit text length
    summary={"query": "Give a detailed summary about it and include people, place, company and industry names as well"},  # Custom summary prompt
)
mlflow.log_param("News", search_results)

print("Search Results:", search_results)

Search Results: Title: Alphabet, Inc. (GOOG) Earnings Date, Call, Release, and History
URL: https://financhill.com/stocks/sp500/goog/earnings
ID: https://financhill.com/stocks/sp500/goog/earnings
Score: None
Published Date: 2025-09-13T00:00:00.000Z
Author: Financhill.com Staff
Image: https://static.financhill.com/1e057e4af20e5835ca2ffe13db60f824580ce97b/images/social.png
Favicon: https://static.financhill.com/1e057e4af20e5835ca2ffe13db60f824580ce97b/images/favicon.png
Extras: None
Subpages: None
Text: [CCL\
\
57\
\
-4](https://financhill.com/stock-score/charts/CCL) [GRMN\
\
53\
\
-3](https://financhill.com/stock-score/charts/GRMN) [JNJ\
\
50\
\
-3](https://financhill.com/stock-score/charts/JNJ) [EL\
\
18\
\
-5](https://financhill.com/stock-score/charts/EL) [FTI\
\
69\
\
-7](https://financhill.com/stock-score/charts/FTI) [MKC\
\
51\
\
-3](https://financhill.com/stock-score/charts/MKC) [ETN\
\
68\
\
3](https://financhill.com/stock-score/charts/ETN) [TMO\
\
55\
\
-7](https://financhill.co

In [63]:
from pydantic import BaseModel
from google import genai
from google.genai.types import GenerateContentConfig
from typing import Literal
import json

class News(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: Literal["Positive","Negative","Neutral"]
    people_names: list
    places_names: list
    other_companies_referred: list
    related_industries: list
    market_implications: str
    confidence_score: int 


client = genai.Client()
response = client.models.generate_content(
    model=model_name,
    contents=f"Summarize this news {search_results} and do the sentiment analysis. Only refer to the information provided and don't add anything by yourself.",
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=News,
    ),
)
mlflow.log_param("Sentiment", response.text)
print(response.text)

{
  "company_name": "Alphabet Inc.",
  "stock_code": "GOOG",
  "newsdesc": "Alphabet Inc. (GOOG) is a leading technology company specializing in internet-related services and products. Its current stock price is near $240. Its next earnings release is scheduled for October 28, 2025. Alphabet reported an earnings per share (EPS) of $2.31 which represents a beat of analyst forecast a $2.20 per share. The 54 analysts providing 12-month price projections for Alphabet, Inc. shares anticipate an average target of $227.62, ranging from a low estimate of $185.00 to a high estimate of $270.00.",
  "sentiment": "Positive",
  "people_names": [
    "Larry Page",
    "Sergey Brin",
    "Sundar Pichai"
  ],
  "places_names": [
    "Mountain View, California",
    "California"
  ],
  "other_companies_referred": [
    "Meta Platforms, Inc.",
    "Netflix, Inc.",
    "Pinterest, Inc.",
    "Snap, Inc.",
    "Adobe",
    "Visa",
    "United Airlines",
    "Amazon",
    "Microsoft",
    "OpenAI"
  ],
  "

Trace(trace_id=tr-ce3d20a339c6633cdd4af90ee272cbfd)